In [1]:
#Allows relative imports
import os
import sys
import pandas as pd 

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
#imports 
from src.preprocessing import *
from src.models import *
from src.train_eval_helpers import *
from src.plots import *
import torch
import torch.nn as nn
import torch.nn.functional as F
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 300
import seaborn as sns
sns.set_style('darkgrid')

#checking gpu status
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using : {}".format(device))
else:
    device = torch.device('cpu')
    print("Using : {}".format(device))

RANGE = range(12,17)
TRAINDIR = '../TrainingData/'

from IPython.display import display_html
def display_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

Using : cuda


In [2]:
#emb2 = pd.read_csv('../training_data_new/emerson_raw/batch2/emerson_clean_batch2_120samples_vdj.tsv', sep='\t', header=0)
emb1 = pd.read_csv('../training_data_new/emerson_raw/batch1/emerson_batch1_626hla_parsed_tagged.tsv', sep='\t', header=0)#, nrows=10000)
sample1 = pd.read_csv('../training_data_new/emerson_raw/batch1/emerson_batch1_sampletags.tsv', 
                      usecols = ['filename', 'pred_cmv', 'true_cmv', 'age', 'sex', 'race', 'hla_a1',
       'hla_a2', 'hla_b1', 'hla_b2', 'class'], header=0, sep='\t')
sample1.head(3)

C:\Users\richi\Anaconda3\envs\pdm\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,filename,pred_cmv,true_cmv,age,sex,race,hla_a1,hla_a2,hla_b1,hla_b2,class
0,HIP00110.tsv,0,0,55,Male,Unknown,A03,A24,B07,Unknown,A03-A24-B07
1,HIP00169.tsv,0,0,41,Male,Unknown,A01,A02,B27,B37,A01-A02-B27-B37
2,HIP00594.tsv,1,1,21,Male,Caucasian,A02,A32,B07,B61,A02-A32-B07-B61


In [18]:
sample1.columns

Index(['filename', 'pred_cmv', 'true_cmv', 'age', 'sex', 'race', 'hla_a1',
       'hla_a2', 'hla_b1', 'hla_b2', 'class', 'A_label', 'B_label'],
      dtype='object')

Comparing Beshnova parsed dataset

In [3]:
og = pd.concat([pd.read_csv('../TrainingData/NormalCDR3.txt', header=None, names=['amino_acid']), pd.read_csv('../TrainingData/NormalCDR3_test.txt', header=None, names=['amino_acid'])])
og.head()

,amino_acid
0,CASSLKPNTEAFF
1,CASSAHIANYGYTF
2,CASSPRPNTEAFF
3,CASTLAGVEETQYF
4,CASSQTSGSYNEQFF


In [4]:
in_og = og.amino_acid.values
query = emb2.query('amino_acid in @in_og').sort_values('frequency',ascending=False)
query.head(3)

,amino_acid,frequency,v_family,v_gene,d_family,d_gene,j_family,j_gene,filename,age,sex,cohort,race,pred_cmv,true_cmv
13696010,CASSPNRGNEQYF,0.067004,TCRBV05,TCRBV05-06,TCRBD01,TCRBD01-01,TCRBJ02,TCRBJ02-07,Keck0115_MC1.tsv,25,Male,2,Caucasian,1,1
12586358,CASYTSSYEQYF,0.042817,TCRBV27,TCRBV27-01,TCRBD02,TCRBD02-01,TCRBJ02,TCRBJ02-07,Keck0108_MC1.tsv,29,Female,2,Black or African American,0,1
3780398,CASSPDRAGTQYF,0.031290,TCRBV13,TCRBV13-01,TCRBD01,TCRBD01-01,TCRBJ02,TCRBJ02-03,Keck0052_MC1.tsv,33,Male,2,Caucasian,1,1


In [7]:
#Trying to compute the class using binary labels.
# Ex for a patient with HLA : 'A01', 'A03', 'B07', 'Unknown', his label should be [1, 0, 1, 0, ... 0] for HLA-A and [0, 1, 0, ... 0] for HLA-B
HLA_A = np.array(sorted(pd.unique(sample1[["hla_a1", "hla_a2"]].values.ravel()), key = str))[:-1] #-1 to drop Unknown.
HLA_B = np.array(sorted(pd.unique(sample1[["hla_b1", "hla_b2"]].values.ravel()), key = str))[:-1] #-1 to drop Unknown

count_A, count_B = pd.DataFrame(), pd.DataFrame()
for val in HLA_A:
    count_A = count_A.append(pd.DataFrame.from_dict({val: len(sample1.query('hla_a1 == @val or hla_a2 == @val'))}, 
                                                orient='index', 
                                                columns=['count']),
                         ignore_index=False)
for val in HLA_B:
    count_B = count_B.append(pd.DataFrame.from_dict({val: len(sample1.query('hla_b1 == @val or hla_b2 == @val'))}, 
                                                orient='index', 
                                                columns=['count']),
                         ignore_index=False)
print(len(HLA_A), len(HLA_B))
display_side(count_A.sort_values('count',ascending=False),count_B.sort_values('count',ascending=False))

21 40


,count
A02,294
A01,186
A03,153
A24,129
A11,61
A29,50
A26,50
A68,47
A32,39
A30,35


##### old getting label draft here

In [ ]:
#Should probably save these 2 as "classes" or pickle them somewhere.
#HLA_A = np.array(sorted(pd.unique(sample1[["hla_a1", "hla_a2"]].values.ravel()), key = str))[:-1] #-1 to drop Unknown.
#HLA_B = np.array(sorted(pd.unique(sample1[["hla_b1", "hla_b2"]].values.ravel()), key = str))[:-1] #-1 to drop Unknown.
with open('hla_a.pkl', 'rb') as f: 
    HLA_A = pickle.load(f)

with open('hla_b.pkl', 'rb') as f: 
    HLA_B = pickle.load(f)
    
def sample_topn(df, n):
    """Group by filename, sort and take top n sequences"""
    grp = df.sort_values('frequency', ascending = False).groupby('filename')
    #Here I take the minimum between 
    sample_map = {k:min(n,v) for (k,v) in zip(sorted(df.filename.unique(), key=str),
                                              grp.size().sort_index().values)}
    return grp.apply(lambda group: group.head(sample_map[group.name])).reset_index(drop=True)

def get_ab_label(row):
    a1 = row['hla_a1']
    a2 = row['hla_a2']
    b1 = row['hla_b1']
    b2 = row['hla_b2']
    A_label = torch.from_numpy(((HLA_A==a1) | (HLA_A == a2)).astype(int))
    B_label = torch.from_numpy(((HLA_B==b1) | (HLA_B == b2)).astype(int))
    row['A_label'] = A_label
    row['B_label'] = B_label
    return row

def get_vdj_label(row):
    v = row['v_gene']
    j = row['j_gene']
    

In [ ]:
sample1 = sample1.apply(get_ab_label,axis=1)
sample1.to_csv('../training_data_new/emerson_raw/batch1/emerson_batch1_sampletags.tsv', 
               sep='\t', header=True, index=False)

In [22]:
emb1.to_csv('../training_data_new/emerson_raw/batch1/emerson_b1_basic_infos_parsed_tagged.tsv',
            columns = ['amino_acid', 'frequency', 'v_gene', 'j_gene', 'filename', 
                       'hla_a1', 'hla_a2', 'hla_b1', 'hla_b2'],
            sep='\t', header = True, index = False)

#### Checking DataLoader

In [23]:
emb1 = pd.read_csv('../training_data_new/emerson_raw/batch1/emerson_b1_basic_infos_parsed_tagged.tsv',
                      usecols = ['amino_acid', 'frequency', 'v_gene', 'j_gene', 'filename', 'hla_a1', 'hla_a2', 'hla_b1', 'hla_b2'],
                   sep='\t', header=0)

In [24]:
emb1.head(3)

,amino_acid,frequency,v_gene,j_gene,filename,hla_a1,hla_a2,hla_b1,hla_b2
0,CATTGTSGGPSQSTQYF,0.010946,TCRBV10-03,TCRBJ02-03,HIP00110.tsv,A03,A24,B07,Unknown
1,CASSLRVGGYGYTF,0.010841,TCRBV07-09,TCRBJ01-02,HIP00110.tsv,A03,A24,B07,Unknown
2,CASRRLGGLNTEAFF,0.006130,TCRBV25-01,TCRBJ01-01,HIP00110.tsv,A03,A24,B07,Unknown


In [20]:
import os
import torch
import pandas as pd
import numpy as np
import math

from src.preprocessing import *
from torch.utils.data import Dataset, DataLoader

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

class emerson_dataset(Dataset):
    """
    Dataset class for data from the Emerson study (batch 1)
    With information on 
    """
    def __init__(self, path):
        emerson = pd.read_csv(path, sep='\t', header=0)
        self.seq = emerson.amino_acid.values
        #self.v_gene = emerson.v_gene.values
        #self.j_gene = emerson.j_gene.values
        self.hla_a1 = emerson.hla_a1.values
        self.hla_a2 = emerson.hla_a2.values
        self.hla_b1 = emerson.hla_b1.values
        self.hla_b2 = emerson.hla_b2.values
        self.patients = emerson.filename.values
        self.len = len(emerson)
        
    def __getitem__(self,index):
        labels = get_all_label(self.hla_a1[index],
                               self.hla_a2[index],
                               self.hla_b1[index],
                               self.hla_b2[index]) #gets (A_labels, B_labels)
        sequence = self.seq[index]
        return sequence, labels
    
    def __len__(self):
        return self.len

    def get_patient(self, patient_filename):
        index = self.patients==patient_filename
        return self.__getitem__(index)

In [63]:
dataset = emerson_dataset('../training_data_new/emerson_raw/batch1/emerson_b1_basic_infos_parsed_tagged.tsv')

In [129]:
patient30 = dataset.get_patient(dataset.patients[30])

In [3]:
import random
def sample_topn(df, n):
    """Group by filename, sort by TCR frequency, and take top n sequences"""
    grp = df.sort_values('frequency', ascending = False).groupby('filename')
    #Here I take the minimum between 
    sample_map = {k:min(n,v) for (k,v) in zip(sorted(df.filename.unique(), key=str),
                                              grp.size().sort_index().values)}
    return grp.apply(lambda group: group.head(sample_map[group.name])).reset_index(drop=True)

def sample_topn_patients(df, top_n, patient_n):
    """Sample patient_n unique patient classes (combination of hla_a1,a2,b1,b2) and the top_n most frequent TCRs"""
    classes = random.sample(sample1['class'].unique().tolist(), patient_n)
    while len(sample1[sample1['class'].isin(classes)]) != patient_n:
        classes = random.sample(sample1['class'].unique().tolist(), patient_n)
    print(classes)
    fns = sample1[sample1['class'].isin(classes)].filename.values
    patient_sample = sample_topn(emb1.query('filename in @fns'), top_n)
    return patient_sample

In [54]:
classes = random.sample(sample1['class'].unique().tolist(), 10)
while len(sample1[sample1['class'].isin(classes)]) != 10:
    classes = random.sample(sample1['class'].unique().tolist(), 10)
classes, len(sample1[sample1['class'].isin(classes)])

(['A02-A24-B15-B55',
  'A02-A30-B18-B39',
  'A01-A68-B08-B14',
  'A02-A28-B35-B53',
  'A02-A68-B08-B44',
  'A02-A31-B07-B35',
  'A24-A30-B13-B61',
  'A29-A30-B13-B57',
  'A02-A03-B08-B51',
  'A01-A32-B08-B60'],
 10)

In [61]:
patient_sample.to_csv('./patient_sample_10k_10patients.tsv', sep='\t', header=True, index=False)

In [4]:
sub = sample_topn(emb1, 10000)

In [6]:
sub.head()
sub.to_csv('../training_data_new/emerson_raw/batch1/emerson_batch1_top10k_hla.tsv',
           sep='\t', header= True, index=False)

#### Splitting emb1 into Train/Test set
Taking 62 (~10%) of the dataset to put into the test set. Sampling based on the "len_classes" (number of HLA resolved)

In [14]:
sample1.groupby('len_class').count()

,filename,pred_cmv,true_cmv,age,sex,race,hla_a1,hla_a2,hla_b1,hla_b2,class
len_class,,,,,,,,,,,
2,15,15,15,15,15,15,15,15,15,15,15
3,102,102,102,102,102,102,102,102,102,102,102
4,509,509,509,509,509,509,509,509,509,509,509


In [52]:
c4 = sample1.query('len_class==4').sample(50)#.50/509 ~0.1
c3 = sample1.query('len_class==3').sample(10)#.10/102 ~0.1
c2 = sample1.query('len_class==2').sample(2)#. 2/15 ~0.13
#Indices
test_indices = c4.index.union(c3.index.union(c2.index))
train_indices = sample1.index.difference(test_indices)
files_train = sample1.loc[train_indices,'filename'].values
files_test = sample1.loc[test_indices,'filename'].values



In [54]:
emb1 = pd.read_csv('../training_data_new/emerson_raw/batch1/emerson_batch1_626hla_parsed_tagged.tsv', sep='\t', header=0)
train = emb1.query('filename in @files_train')
test = emb1.query('filename in @files_test')

In [57]:
train.to_csv('../training_data_new/emerson_raw/batch1/emerson_batch1_train.tsv', sep='\t',
             header = True, index = False)
test.to_csv('../training_data_new/emerson_raw/batch1/emerson_batch1_test.tsv', sep='\t',
            header = True, index = False)

In [12]:

sub_test = sample_topn(test, 10000)
sub_train = sample_topn(train, 10000)

sub_train.to_csv('../training_data_new/emerson_raw/batch1/emerson_batch1_train_top10k_hla.tsv',
           sep='\t', header= True, index=False)
sub_test.to_csv('../training_data_new/emerson_raw/batch1/emerson_batch1_test_top10k_hla.tsv',
           sep='\t', header= True, index=False)